In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

from caption_lda_model_train import mark_captions, create_tokenizer
from evaluation.caption_category_model_eval import generate_predictions, beam_search_predictions
from models.caption_lda_model import create_model
from dataset.utils import print_progress_bar, load_image

In [2]:
def load_data(data_type, data_dir):
    # Path for the cache-file.
    feature_cache_path = os.path.join(
        data_dir, 'feature_transfer_values_{}.pkl'.format(data_type)
    )
    topics_cache_path = os.path.join(
        data_dir, 'topics16_{}.pkl'.format(data_type)
    )
    captions_cache_path = os.path.join(
        data_dir, 'captions_{}.pkl'.format(data_type)
    )

    feature_path_exists = os.path.exists(feature_cache_path)
    topic_path_exists = os.path.exists(topics_cache_path)
    caption_path_exists = os.path.exists(captions_cache_path)
    if feature_path_exists and topic_path_exists and caption_path_exists:
        with open(feature_cache_path, mode='rb') as file:
            feature_obj = pickle.load(file)
        with open(topics_cache_path, mode='rb') as file:
            topics = pickle.load(file)
        with open(captions_cache_path, mode='rb') as file:
            captions = pickle.load(file)
    else:
        sys.exit('processed {} data does not exist.'.format(data_type))

    print('{} data loaded from cache-file.'.format(data_type))
    return feature_obj, topics, captions

In [3]:
data_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'dataset', 'processed_lda_data')

In [4]:
# Load pre-processed data
features_train, topics_train, captions_train = load_data(
    'train', data_dir
)

train data loaded from cache-file.


In [5]:
# Load test data
with open(data_dir + '/feature_transfer_values_test.pkl', mode='rb') as file:
    features_test = pickle.load(file)
with open(data_dir + '/captions_test.pkl', mode='rb') as file:
    captions_test = pickle.load(file)
with open(data_dir + '/images_test.pkl', mode='rb') as file:
    images_test = pickle.load(file)

In [6]:
def load_test_topics(features_train, topics_train, features_test):
    topics_test = []
    print_progress_bar(0, len(features_test))
    for idx, feature in enumerate(features_test):
        topic_idx = np.argmax(np.sum(np.abs(features_train - feature), axis=1))
        topics_test.append(topics_train[topic_idx])
        print_progress_bar(idx + 1, len(features_test))
    return np.array(topics_test).astype(topics_train.dtype)

In [ ]:
topics_test = load_test_topics(features_train, topics_train, features_test)

In [7]:
# create tokenizer
mark_start = 'startseq'
mark_end = 'endseq'
captions_train_marked = mark_captions(captions_train, mark_start, mark_end)
tokenizer, vocab_size = create_tokenizer(captions_train_marked)

# Get data size
max_tokens = 16

In [8]:
# Embedding vector file
glove_path = os.path.join(os.path.dirname(os.path.abspath('')), 'dataset', 'glove.6B.300d.txt')

# Create Model
model = create_model(
    topics_train.shape[1:],
    features_test.shape[1:],
    tokenizer.word_index,
    glove_path,
    mark_start,
    mark_end,
    vocab_size,
    max_tokens
)

Creating word to vec map...
Done!
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input (InputLayer)      (None, 16)           0                                            
__________________________________________________________________________________________________
topic_input (InputLayer)        (None, 16)           0                                            
__________________________________________________________________________________________________
feature_input (InputLayer)      (None, 4096)         0                                            
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, 16, 300)      2593500     caption_input[0][0]              
___________________________________________________________________________

In [9]:
# Weights path
weights_path = os.path.join(os.path.dirname(os.path.abspath('')), 'weights', 'cplda-weights-04-2.59-b128-t16.hdf5')

# Load weights
try:
    model.load_weights(weights_path)
    print('Weights loaded.')
except Exception as e:
    print('Error loading the weights.')
    print(e)

Weights loaded.


In [ ]:
idx = 30

y_pred = generate_predictions(
    topics_test[idx],
    features_test[idx],
    model,
    tokenizer,
    mark_start,
    mark_end,
    max_tokens
)

# Display image
img_path = os.path.join(os.path.dirname(os.path.abspath('')), 'dataset', images_test[idx])
img = load_image(img_path)
plt.imshow(img)
plt.show()

# Pred caption
print(y_pred, end='\n')

# original caption
for cap in captions_test[idx]:
    print(cap)

In [25]:
t = []
for idx, feature in enumerate(features_test[145:150]):
    topic_idx = np.argmax(np.sum(np.abs(features_train - feature), axis=1))
    print(topic_idx)
    t.append(topics_train[topic_idx])

14117
14117
10522
10522
14117


In [11]:
t[0] - t[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
a = np.sum(np.abs(features_train - features_test[0]), axis=1)
a

array([401.2898 , 287.10602, 388.63373, ..., 297.1723 , 545.22205,
       420.00818], dtype=float32)

In [19]:
b = np.sum(np.abs(features_train - features_test[148]), axis=1)
b

array([409.01874, 504.88245, 604.2456 , ..., 531.49133, 470.52087,
       485.12753], dtype=float32)

In [24]:
print(np.argmax(b))

10522

In [20]:
a[14117]

845.2254

In [21]:
b[14117]

795.79736

In [22]:
for i in b:
    if i > 800:
        print(i)

832.63403
